# Ejercicios Semana 5

In [34]:
import json
import requests

## Problemas de Archivos

Dado el archivo de texto files/SalesJan2009.csv:

1. Procese el archivo para obtener las compras realizadas en un país dado.
2. Procese el archivo para obtener las compras realizadas con un medio de pago dado.

In [23]:
def read_csv(path):
    data = []
    with open(path, 'r', encoding="utf-8") as f:
        data = f.readlines()
    rows = [row.split(',') for row in data]
    head = rows.pop(0)
    elementos = []
    for row in rows:
        d = {}
        for indice, atributo in enumerate(head):
            d[atributo] = row[indice]
        elementos.append(d)
    return elementos

In [24]:
def extract_values(value, match, attribute, items):
    find = []
    for i in items:
        if value == i[match]:
            find.append(i[attribute])
    return find

In [29]:
filtered = extract_values('France', 'Country', 'Price', read_csv('files/SalesJan2009.csv'))
filtered[:2]

['1200', '1200']

In [26]:
sum([float(p) for p in filtered])

53100.0

In [30]:
filtered = extract_values('Diners', 'Payment_Type', 'Price', read_csv('files/SalesJan2009.csv'))
filtered[:2]

['1200', '1200']

In [31]:
sum([float(p) for p in filtered])

133800.0

## Problemas JSON

Usando el API [JSONPlaceholder](https://jsonplaceholder.typicode.com/guide/), realizar las siguientes operaciones

1. Consultar los usuarios y crear una lista en donde se muestre el Identificador y el nombre como una cadena de caracteres.
2. Consultar todos los posts y crear una lista de diccionarios en donde solo se presente el identificador y el titulo.
3. Capturar un identificador ingresado por el usuario y verificar que exista en la lista de usuarios disponibles consultada en el punto 1 y mostrar su información relevante.
4. Con el identificador capturado en el punto 3 identificar cuales posts de los consultados en el punto dos le pertenecen al usuario y crear un nuevo diccionario en donde este la información del usuario y un listado de sus posts.
5. Guardar el resultado del punto 5 en un archivo JSON
6. Capturar un identificador ingresado por el usuario y verificar que exista en la lista de post que se consulto en el punto 2 y consultar los comentarios de este post 

In [39]:
def consultar_usuarios():
    response = requests.get('https://jsonplaceholder.typicode.com/users')
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception('No se pudieron consultar los usuarios')
        return []


In [45]:
def consultar_posts():
    response = requests.get('https://jsonplaceholder.typicode.com/posts')
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception('No se pudieron consultar los posts')
        return []

In [47]:
def convertir_posts(posts):
    items = []
    for p in posts:
        items.append({
            'id': p['id'],
            'title': p['title']
        })
    return items

In [53]:
consultar_usuarios()[:2]

[{'id': 1,
  'name': 'Leanne Graham',
  'username': 'Bret',
  'email': 'Sincere@april.biz',
  'address': {'street': 'Kulas Light',
   'suite': 'Apt. 556',
   'city': 'Gwenborough',
   'zipcode': '92998-3874',
   'geo': {'lat': '-37.3159', 'lng': '81.1496'}},
  'phone': '1-770-736-8031 x56442',
  'website': 'hildegard.org',
  'company': {'name': 'Romaguera-Crona',
   'catchPhrase': 'Multi-layered client-server neural-net',
   'bs': 'harness real-time e-markets'}},
 {'id': 2,
  'name': 'Ervin Howell',
  'username': 'Antonette',
  'email': 'Shanna@melissa.tv',
  'address': {'street': 'Victor Plains',
   'suite': 'Suite 879',
   'city': 'Wisokyburgh',
   'zipcode': '90566-7771',
   'geo': {'lat': '-43.9509', 'lng': '-34.4618'}},
  'phone': '010-692-6593 x09125',
  'website': 'anastasia.net',
  'company': {'name': 'Deckow-Crist',
   'catchPhrase': 'Proactive didactic contingency',
   'bs': 'synergize scalable supply-chains'}}]

In [51]:
convertir_posts(consultar_posts())[:2]

[{'id': 1,
  'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit'},
 {'id': 2, 'title': 'qui est esse'}]

Alternativa Lambda Function

In [52]:
list(map(lambda p: {'id': p['id'], 'title':p['title']}, consultar_posts()))[:2]

[{'id': 1,
  'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit'},
 {'id': 2, 'title': 'qui est esse'}]

In [73]:
class UsuarioNoEncontrado(Exception):
    def __init__(self, userId):
        self.userId = userId
    def __str__(self):
        return f"El usuario: {self.userId} no ha sido encontrado"

In [93]:
def consultar_usuario():
    try:
        userId = int(input('Ingrese el id del usuario que desea buscar: '))
        print(userId)
        usuarios = consultar_usuarios()
        usuario = None
        for u in usuarios:
            if u['id'] == userId:
                usuario = u
                break
        if not usuario:
            raise UsuarioNoEncontrado(userId)
        print(f'Nombre: {usuario["name"]}')
        print(f'Username: {usuario["username"]}')
        print(f'Correo: {usuario["email"]}')
    except ValueError:
        print('Ingreso un id no valido')
        return None
    except UsuarioNoEncontrado as e:
        print(e)
        return None
    else:
        return usuario


In [83]:
consultar_usuario()

2
Nombre: Ervin Howell
Username: Antonette
Correo: Shanna@melissa.tv


2

In [104]:
def encontrar_posts():
    try:
        usuario = consultar_usuario()
        if not usuario:
            raise UsuarioNoEncontrado()
        posts = consultar_posts()

        filtrados = list(filter(lambda p:p['userId'] == usuario['id'], posts))
        copia = usuario.copy()
        copia['posts'] = filtrados
        return copia
    except UsuarioNoEncontrado as e:
        print(e)
        return None

In [106]:
user = encontrar_posts()

2
Nombre: Ervin Howell
Username: Antonette
Correo: Shanna@melissa.tv


In [109]:
def guardar_usuario():
    data = encontrar_posts()
    with open('json/usuario.json', 'w') as f:
        f.write(json.dumps(data, indent=4))

In [113]:
def consultar_comentarios_post(postId):
    response = requests.get(f'https://jsonplaceholder.typicode.com/posts/{postId}/comments')
    if response.status_code != 200:
        raise Exception(f'No se pudo consultar los comentarios del post: {postId}')
    comments = response.json()
    return comments
    

In [122]:
def consultar_comentarios():
    try:
        postId = int(input('Ingrese el id del post que desea consultar'))
        posts = consultar_posts()
        post = None
        for p in posts:
            if p['id'] == postId:
                post = p
                break
        if not post:
            raise Exception('Post no encontrado')
        comentarios = consultar_comentarios_post(post['id'])
        return comentarios
    except ValueError:
        print('El id del post es invalido')
        return []
    except Exception as e:
        print(e)
        return []
    


In [124]:
consultar_comentarios()

[{'postId': 15,
  'id': 71,
  'name': 'vel pariatur perferendis vero ab aut voluptates labore',
  'email': 'Lavinia@lafayette.me',
  'body': 'mollitia magnam et\nipsum consequatur est expedita\naut rem ut ex doloremque est vitae est\ncumque velit recusandae numquam libero dolor fuga fugit a'},
 {'postId': 15,
  'id': 72,
  'name': 'quia sunt dolor dolor suscipit expedita quis',
  'email': 'Sabrina.Marks@savanah.name',
  'body': 'quisquam voluptas ut\npariatur eos amet non\nreprehenderit voluptates numquam\nin est voluptatem dicta ipsa qui esse enim'},
 {'postId': 15,
  'id': 73,
  'name': 'ut quia ipsa repellat sunt et sequi aut est',
  'email': 'Desmond_Graham@kailee.biz',
  'body': 'nam qui possimus deserunt\ninventore dignissimos nihil rerum ut consequatur vel architecto\ntenetur recusandae voluptate\nnumquam dignissimos aliquid ut reprehenderit voluptatibus'},
 {'postId': 15,
  'id': 74,
  'name': 'ut non illum pariatur dolor',
  'email': 'Gussie_Kunde@sharon.biz',
  'body': 'non a

## Excepciones

1. ArithmeticError
    * ZeroDivisionError
2. TypeError
3. ValueError
4. LookupError
    * IndexError
    * KeyError
5. NameError
6. Import Error
    * ModuleNotFoundError
7. OSError
    * FileNotFoundError
    * IsADirectoryError

[Documentación Oficial Python Excepciones](https://docs.python.org/es/3/library/exceptions.html)

In [120]:
try:
    lista.append(5)
except NameError:
    print('Variable no definida')

Variable no definida


In [121]:
with open('juan.txt', 'r') as f:
    print('juan')

FileNotFoundError: [Errno 2] No such file or directory: 'juan.txt'

In [ ]:
with open('base', 'r') as f:
    print('acceder a base')

IsADirectoryError: [Errno 21] Is a directory: 'base'

In [ ]:
class MiExcepcion(Exception):
    def __init__(self, mensaje, informacion):
        self.mensaje = mensaje
        self.informacion = informacion
    
try:
    raise MiExcepcion("Mi Mensaje",2)
except MiExcepcion as e:
    print(e.mensaje)
    print(e.informacion)

Mi Mensaje
2
